In [1]:
# Initialize
import numpy as np
import pandas as pd
import re

In [2]:
peptides_file = 'pd22_exports/pd22_export_exp1_Heart_DKO-Sirt3-Comparison_PeptideGroups.txt'
proteins_file = 'pd22_exports/pd22_export_exp1_Heart_DKO-Sirt3-Comparison_Proteins.txt'
pep = pd.read_csv(peptides_file, sep='\t', low_memory=False)
pro = pd.read_csv(proteins_file, sep='\t', low_memory=False)

# Import data from proteome discoverer

For the peptides dataframe, use the first listed Master Protein Accession ID as the primary identifier. This identifier will be matched in the Proteins dataframe and used to pull additional information about the peptide. 

In [3]:
pep['Accession'] = pep['Master Protein Accessions'].str.split('; ', expand=True)[0]
pro['Entrez'] = pro['Entrez Gene ID'].str.split('; ', expand=True)[0]
pro['Ensembl'] = pro['Ensembl Gene ID'].str.split('; ', expand=True)[0]

pep = pep.merge(pro[['Accession', 'Description', 'Entrez', 'Gene Symbol']], on='Accession')

Next, import a pre-generated dataframe (stored as a pickled binary format). The information is based on the MitoCarta2 database, pulled from MitoMiner (http://mitominer.mrc-mbu.cam.ac.uk/release-4.0/begin.do) with additional, manually curated information about the particular location within the mitochondria. This will be used to identify peptides and proteins as mitochondrial.

IMS = intermembrane space

matrix = mitochondrial matrix

In [4]:
mitocarta_df = pd.read_csv('mitocarta.csv')

pep['MitoCarta2_List'] = pep.Entrez.isin(mitocarta_df.MouseGeneID.astype(str))
pep['Matrix'] = pep.Entrez.isin(mitocarta_df.MouseGeneID[mitocarta_df.Matrix].astype(str))
pep['IMS'] = pep.Entrez.isin(mitocarta_df.MouseGeneID[mitocarta_df.IMS].astype(str))

pro['MitoCarta2_List'] = pro.Entrez.isin(mitocarta_df.MouseGeneID.astype(str))
pro['Matrix'] = pro.Entrez.isin(mitocarta_df.MouseGeneID[mitocarta_df.Matrix].astype(str))
pro['IMS'] = pro.Entrez.isin(mitocarta_df.MouseGeneID[mitocarta_df.IMS].astype(str))

Next, we will identify columns that we want to store in a "Master Index". This will contian key information we want to retain from the proteome discoverer 2.2 exported file

In [5]:
# a list to of columns to make a master index
protein_mi_list = ['Accession', 
                   'Description',
                   'Entrez',
                   'Gene Symbol',
                   'Ensembl',
                   'MitoCarta2_List',
                   'Matrix', 
                   'IMS',
                   'Coverage [%]',
                   '# Peptides',
                   '# PSMs',
                   '# Protein Unique Peptides',
                   '# Unique Peptides', 
                   'Chromosome',
                   'Modifications',
                   'Acetyl (K) Count',
                   'Acetyl (K) Positions']

peptide_mi_list = ['Accession', 
                   'Description',
                   'Entrez',
                   'Gene Symbol',
                   'Modifications', 
                   'MitoCarta2_List',
                   'Matrix', 
                   'IMS', 
                   'Modifications in Proteins',
                   'Modifications in Proteins (all Sites)',
                   'Positions in Proteins',]

Now, we identify the abundance columns from the experiment. These contain the quantification data for each TMT channel. There are 2 fractions that are mentioned:

- F1 = "Input", the unenriched, pooled TMT-labeled peptides for all 10 samples. 
- F2 = "Acetyl", the Acetyl-lysine IP-enriched fraction 

The numbers (126, 130C) indicate the specific TMT tag associated with the column. 

Genotypes:
- S3FL = Sirt3 floxed control
- S3KO = Sirt3 knockout (via MCK-Cre)
- DFC = Dual flox control (Sirt3 and Crat floxed)
- DKO = Dual Knock-out (Sirt3 and Crat knockout via MCK-Cre)

In [6]:
input_abundance_list = ['Abundance: F1: 126, Sample, Input (Fraction), S3KO (Genotype)',
                         'Abundance: F1: 127C, Sample, Input (Fraction), S3KO (Genotype)',
                         'Abundance: F1: 129N, Sample, Input (Fraction), S3KO (Genotype)',
                         'Abundance: F1: 127N, Sample, Input (Fraction), S3FL (Genotype)',
                         'Abundance: F1: 130N, Sample, Input (Fraction), S3FL (Genotype)',
                         'Abundance: F1: 128N, Sample, Input (Fraction), DFC (Genotype)',
                         'Abundance: F1: 129C, Sample, Input (Fraction), DFC (Genotype)',
                         'Abundance: F1: 128C, Sample, Input (Fraction), DKO (Genotype)',
                         'Abundance: F1: 130C, Sample, Input (Fraction), DKO (Genotype)',
                         'Abundance: F1: 131, Sample, Input (Fraction), DKO (Genotype)',]

acetyl_abundance_list = ['Abundance: F2: 126, Sample, Acetyl (Fraction), S3KO (Genotype)',
                         'Abundance: F2: 127C, Sample, Acetyl (Fraction), S3KO (Genotype)',
                         'Abundance: F2: 129N, Sample, Acetyl (Fraction), S3KO (Genotype)',
                         'Abundance: F2: 127N, Sample, Acetyl (Fraction), S3FL (Genotype)',
                         'Abundance: F2: 130N, Sample, Acetyl (Fraction), S3FL (Genotype)',
                         'Abundance: F2: 128N, Sample, Acetyl (Fraction), DFC (Genotype)',
                         'Abundance: F2: 129C, Sample, Acetyl (Fraction), DFC (Genotype)',
                         'Abundance: F2: 128C, Sample, Acetyl (Fraction), DKO (Genotype)',
                         'Abundance: F2: 130C, Sample, Acetyl (Fraction), DKO (Genotype)',
                         'Abundance: F2: 131, Sample, Acetyl (Fraction), DKO (Genotype)',]

These are shorter, more friendly versions of those identifier that will be used to rename the columns

In [7]:
input_consolidated_list = ['S3KO,126,Input',
                           'S3KO,127C,Input',
                           'S3KO,129N,Input',
                           'S3FL,127N,Input',
                           'S3FL,130N,Input',
                           'DFC,128N,Input',
                           'DFC,129C,Input',
                           'DKO,128C,Input',
                           'DKO,130C,Input',
                           'DKO,131,Input']

acetyl_consolidated_list = ['S3KO,126,Acetyl',
                            'S3KO,127C,Acetyl',
                            'S3KO,129N,Acetyl',
                            'S3FL,127N,Acetyl',
                            'S3FL,130N,Acetyl',
                            'DFC,128N,Acetyl',
                            'DFC,129C,Acetyl',
                            'DKO,128C,Acetyl',
                            'DKO,130C,Acetyl',
                            'DKO,131,Acetyl']

In [8]:
acetyl_columns_df = pd.DataFrame.from_records([col.split(',') for col in acetyl_consolidated_list], 
                                              columns=['Genotype', 'Channel', 'Fraction'])

input_columns_df = pd.DataFrame.from_records([col.split(',') for col in input_consolidated_list], 
                                              columns=['Genotype', 'Channel', 'Fraction'])

acetyl_columns_df.set_index(acetyl_columns_df.columns.tolist(), inplace=True)
input_columns_df.set_index(input_columns_df.columns.tolist(), inplace=True)

## Locate the Acetyl Peptides (Kac)

We look for peptides that were quantified from the Acetyl fraction AND contain an identified actyl-lysine residue present. While the acetyl fraction is enriched for acetylated lysine residues, not all identified peptides within this fraction contain this post-translational modification. 

In [9]:
acetyl_df = pep[pep['Modifications'].str.contains('Acetyl')][acetyl_abundance_list].dropna()
acetyl_df.columns = acetyl_columns_df.index

# Flatten the column headers for working in R
acetyl_df.columns = ['_'.join(col).strip() for col in acetyl_df.columns.values]

In [10]:
# Define the input - i.e., everything identified in the non-enriched fraction
input_df = pep[input_abundance_list].dropna()
input_df.columns = input_columns_df.index

# Flatten the column headers for symmetry with acetyl
input_df.columns = ['_'.join(col).strip() for col in input_df.columns.values]

## Locate the protein abundances
- Use values obtained from the input fraction - these represent the total proteome
- Only select proteins that are called "Master Proteins" by proteome discoverer 2.2 wiht a FDR < 0.01

In [11]:
protein_df = (pro[(pro.Master=='IsMasterProtein')
                  &(pro['Exp. q-value: Combined']<=0.01)]
              [input_abundance_list]
              .dropna(how='all')
              .copy())

# copy the input headers and rename for proteins
protein_df.columns = input_df.columns
protein_df.rename(columns=lambda x: x.replace('Input', 'Proteins'), inplace=True)


# Normalize Data
- Correct for variations in total protein loading per kit
- use the mean of the sums of total peptide loading (input fraction) to generate a correction factor
- apply this correction factor to the acetyl peptides, 'input fraction', and proteins

## Calculate the scaling factor for each plex 

In [12]:
channel_sums = input_df.sum(axis=0)

In [13]:
print('Individual Channel, Loading relative to the mean')

channel_sums.divide(channel_sums.mean())

Individual Channel, Loading relative to the mean


S3KO_126_Input     1.066501
S3KO_127C_Input    1.056243
S3KO_129N_Input    0.918495
S3FL_127N_Input    1.041513
S3FL_130N_Input    1.012348
DFC_128N_Input     1.107858
DFC_129C_Input     0.830616
DKO_128C_Input     0.909889
DKO_130C_Input     1.002346
DKO_131_Input      1.054191
dtype: float64

In [14]:
print('Scaling factor to adjust relative loading to 1')

scaling_factor = 1/channel_sums.divide(channel_sums.mean())
scaling_factor

Scaling factor to adjust relative loading to 1


S3KO_126_Input     0.937645
S3KO_127C_Input    0.946752
S3KO_129N_Input    1.088738
S3FL_127N_Input    0.960142
S3FL_130N_Input    0.987803
DFC_128N_Input     0.902643
DFC_129C_Input     1.203925
DKO_128C_Input     1.099035
DKO_130C_Input     0.997660
DKO_131_Input      0.948595
dtype: float64

## Apply the Scaling Factor

In [15]:
input_norm = (input_df * scaling_factor)
input_norm.columns = [str(col) + '_norm' for col in input_norm.columns]
input_norm.sum()

S3KO_126_Input_norm     4519881.01
S3KO_127C_Input_norm    4519881.01
S3KO_129N_Input_norm    4519881.01
S3FL_127N_Input_norm    4519881.01
S3FL_130N_Input_norm    4519881.01
DFC_128N_Input_norm     4519881.01
DFC_129C_Input_norm     4519881.01
DKO_128C_Input_norm     4519881.01
DKO_130C_Input_norm     4519881.01
DKO_131_Input_norm      4519881.01
dtype: float64

In [16]:
protein_norm = (protein_df * scaling_factor.rename(lambda x: x.replace('Input', 'Proteins')))
protein_norm.columns = [str(col) + '_norm' for col in protein_norm.columns]
protein_norm.sum()

S3KO_126_Proteins_norm     3.368394e+06
S3KO_127C_Proteins_norm    3.355619e+06
S3KO_129N_Proteins_norm    3.366916e+06
S3FL_127N_Proteins_norm    3.373693e+06
S3FL_130N_Proteins_norm    3.388580e+06
DFC_128N_Proteins_norm     3.372285e+06
DFC_129C_Proteins_norm     3.350314e+06
DKO_128C_Proteins_norm     3.407567e+06
DKO_130C_Proteins_norm     3.388112e+06
DKO_131_Proteins_norm      3.392775e+06
dtype: float64

In [17]:
acetyl_norm = acetyl_df * scaling_factor.rename(lambda x: x.replace('Input', 'Acetyl'))
acetyl_norm.columns = [str(col) + '_norm' for col in acetyl_norm.columns]
acetyl_norm.sum()

S3KO_126_Acetyl_norm     31099.821915
S3KO_127C_Acetyl_norm    30695.293585
S3KO_129N_Acetyl_norm    32907.544787
S3FL_127N_Acetyl_norm    17414.186533
S3FL_130N_Acetyl_norm    16294.502693
DFC_128N_Acetyl_norm     16131.940891
DFC_129C_Acetyl_norm     17898.273591
DKO_128C_Acetyl_norm     43946.024195
DKO_130C_Acetyl_norm     47172.138868
DKO_131_Acetyl_norm      47682.056225
dtype: float64

# Calculate the "relative occupancy" for each plex
- This adjusts the acetyl peptide quantification based on changes in protein quantification. This corrects not for experimental loading, but for genetic changes that may occur between the genetic knock-out and the flox control.
- The relative occupancy corrections will be almost un-noticed in these animals, however. 

In [18]:
# convert the input-normalized data into the log2 space
# subtract the row mean from each column to center each row around 0 (in log 2)
acetyl_norm_log2 = np.log2(acetyl_norm).sub(np.log2(acetyl_norm).mean(axis=1), axis='index')
input_norm_log2 = np.log2(input_norm).sub(np.log2(input_norm).mean(axis=1), axis='index')
protein_norm_log2 = np.log2(protein_norm).sub(np.log2(protein_norm).mean(axis=1), axis='index')


# rename columns to save the information
acetyl_norm_log2.columns = [str(col) + '_log2' for col in acetyl_norm_log2.columns]
input_norm_log2.columns = [str(col) + '_log2' for col in input_norm_log2.columns]
protein_norm_log2.columns = [str(col) + '_log2' for col in protein_norm_log2.columns]


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log2
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log2
  after removing the cwd from sys.path.


Combine the the "raw", load-normalized, and log2-transformed data into a single dataframe

In [19]:
acetyl = pd.concat([pep.loc[acetyl_norm.index][peptide_mi_list], 
                    acetyl_df, 
                    acetyl_norm, 
                    acetyl_norm_log2,], 
                   axis=1)

inputs = pd.concat([pep.loc[input_norm.index][peptide_mi_list], 
                    input_df, 
                    input_norm, 
                    input_norm_log2], 
                   axis=1)

protein = pd.concat([pro.loc[protein_df.index][protein_mi_list], 
                     protein_df, 
                     protein_norm, 
                     protein_norm_log2], 
                    axis=1)


## Calculate the RO
- Merge protein information along with acetyl information into a single dataframe
- The protein abundance information will be used to correc the acetyl data (the relative occupancy)

In [20]:
protein_merge_cols = protein.filter(regex='_log2').columns.tolist() + ['Accession']

In [21]:
merged_acetyl = acetyl.merge(protein[protein_merge_cols], 
                             on='Accession', 
                             how='left', 
                             validate='m:1')

The relative occupancy of the post-translational modification is calculated by subtracting changes in the protien abundance from the acetyl-peptide abundance. For example, if a PTM site is found to be increasing by 2 fold, but the protein abundnace is also increasing by 2 fold, then the relative occupancy would be 0. 

In [22]:
# calculate relative occupancy in acetyl peptides

relative_occupancy = (merged_acetyl.filter(regex='Acetyl_norm_log2')
                      .sub(merged_acetyl.filter(regex='Proteins_norm_log2')
                           # rename the protein column headers to enable easy dataframe subtraction
                           .rename(columns=lambda x: x.replace('Proteins', 'Acetyl'))))

relative_occupancy.rename(columns=lambda x: x+'_ro', inplace=True)

relative_occupancy.head()

,S3KO_126_Acetyl_norm_log2_ro,S3KO_127C_Acetyl_norm_log2_ro,S3KO_129N_Acetyl_norm_log2_ro,S3FL_127N_Acetyl_norm_log2_ro,S3FL_130N_Acetyl_norm_log2_ro,DFC_128N_Acetyl_norm_log2_ro,DFC_129C_Acetyl_norm_log2_ro,DKO_128C_Acetyl_norm_log2_ro,DKO_130C_Acetyl_norm_log2_ro,DKO_131_Acetyl_norm_log2_ro
0,1.060937,-0.107952,0.102072,-0.320426,-0.104683,-1.992402,-0.375107,0.516447,0.598744,0.622369
1,0.211162,-0.049224,0.366715,-0.771666,-0.891082,-0.937294,-0.792389,0.979806,0.896988,0.986983
2,0.540714,0.858363,0.429939,-1.035488,-1.104956,-1.094247,-1.427995,0.659270,1.043544,1.130856
3,0.857344,0.428944,0.577243,-0.705415,-1.047717,-1.406975,-2.076860,0.896228,1.296531,1.180675
4,0.851653,0.736769,0.236516,-1.155680,-0.847551,-1.721933,-1.484057,0.833412,1.195101,1.355769


In [23]:
# stick the relative occupancy data onto the merged data
merged_acetyl = pd.concat([merged_acetyl, relative_occupancy], axis=1)

# drop the protein columns out of the data
merged_acetyl.drop(columns=merged_acetyl.filter(regex='_Protein').columns.tolist(), 
                   inplace=True)

# *limma* analysis

## Export Files for *limma* analysis

It is probably just as easy to do this entire analysis in R, with the limma component incorporated into the data processing. I'm not very experienced in R, so the disjointed analysis with R and python was faster for me. Exporting the data from Python into a CSV to import into R was the quickest method for my purposes. One could also try the rpy2 library (I tinkered with it, but found it to be too much effort for the payoff) or exporting the dataframes from pandas into a feather/arrow format.

In [24]:
acetyl_norm.to_csv('limma/limma_export_dko-s3_kac.csv')

merged_acetyl.filter(regex='_ro$').to_csv('limma/limma_export_dko-s3_RO_kac.csv')

protein.filter(regex='.+_norm$').to_csv('limma/limma_export_dko-s3_proteins.csv')

## Read Files from *limma* analysis

In [25]:
kac_limma = pd.read_csv('limma/eb_fit_dko-s3_kac.tsv', sep='\t')
kac_ro_limma = pd.read_csv('limma/eb_fit_dko-s3_RO_kac.tsv', sep='\t')
prot_limma = pd.read_csv('limma/eb_fit_dko-s3_proteins.tsv', sep='\t')

In [26]:
# merge back onto data
acetyl = pd.concat([acetyl, kac_limma], axis=1)

protein = pd.concat([protein, prot_limma], axis=1)

merged_acetyl = pd.concat([merged_acetyl, kac_ro_limma], axis=1)

# rename to somethign more fitting now
ro_acetyl = merged_acetyl

Rename the limma results to have more intuitive names

- `Coef` = Log2 Fold Change comparison between two groups
- `Res` = Significance. 
    - 0 = not significant
    - 1 = significant difference increasing
    - -1 = significant difference decreasing

In [27]:
limma_match = ['Coef\.', 'p\.value\.', 't\.', 'Res\.']
limma_replace = ['_Log2FC', '_p_value', '_t', '_significant']

for match, replace in zip(limma_match, limma_replace):
    acetyl.rename(columns=lambda x: re.sub(match + '(.+)', '\g<1>' + replace, x), inplace=True)
    protein.rename(columns=lambda x: re.sub(match + '(.+)', '\g<1>' + replace, x), inplace=True)
    ro_acetyl.rename(columns=lambda x: re.sub(match + '(.+)', '\g<1>' + replace, x), inplace=True)

Pull out useful information from the Description field into separate columns. Although there is a Gene Symbol field present, it isn't always populated and isn't always the same

In [28]:
acetyl['Name'] = acetyl.Description.str.split(' [OSGNPESV]{2}=', expand=True)[0]
acetyl['Symbol'] = acetyl.Description.str.split(' [OSGNPESV]{2}=', expand=True)[2]

protein['Name'] = protein.Description.str.split(' [OSGNPESV]{2}=', expand=True)[0]
protein['Symbol'] = protein.Description.str.split(' [OSGNPESV]{2}=', expand=True)[2]

ro_acetyl['Name'] = ro_acetyl.Description.str.split(' [OSGNPESV]{2}=', expand=True)[0]
ro_acetyl['Symbol'] = ro_acetyl.Description.str.split(' [OSGNPESV]{2}=', expand=True)[2]

# reorder the columns so the information
acetyl = acetyl[acetyl.columns.tolist()[-2:] + acetyl.columns.tolist()[:-2]]
protein = protein[protein.columns.tolist()[-2:] + protein.columns.tolist()[:-2]]
ro_acetyl = ro_acetyl[ro_acetyl.columns.tolist()[-2:] + ro_acetyl.columns.tolist()[:-2]]

Sort the data

In [29]:
protein.sort_values('Acetyl (K) Count', ascending=False, inplace=True)
acetyl.sort_values('DKOvsDFC_Log2FC', ascending=False, inplace=True)
ro_acetyl.sort_values('DKOvsDFC_Log2FC', ascending=False, inplace=True)

# Export the Data

In [30]:
writer = pd.ExcelWriter('processed_files/Exp1_DKOvsSirt3_10plex.xlsx', options={'strings_to_numbers': True})

ro_acetyl.to_excel(writer, sheet_name='Exp1_DKOvsSirt3_RO_Kac', index=False)
acetyl.to_excel(writer, sheet_name='Exp1_DKOvsSirt3_Kac', index=False)
protein.to_excel(writer, sheet_name='Exp1_DKOvsSirt3_Protein', index=False)

writer.save()

In [31]:
ro_acetyl.to_csv('processed_files/Exp1_DKOvsSirt3_RO_Kac.csv', index=False)
acetyl.to_csv('processed_files/Exp1_DKOvsSirt3_Kac.csv', index=False)
protein.to_csv('processed_files/Exp1_DKOvsSirt3_Protein.csv', index=False)

# Top 25 Fold Changes

This is one (simple) method to assess the severity of hyperacetylation. We'll only look at the relative occupancy corrected acetylpeptide abundances for this analysis

First, we will limit our analysis to the mitochondria as defined by the mitocarta2 dataset

In [32]:
df = ro_acetyl[ro_acetyl.MitoCarta2_List==True].copy()

Now, we will move out of the log2 space for more intuitive comparisons

In [33]:
df['DKOvsDFC_FC'] = np.exp2(df.DKOvsDFC_Log2FC)
df['S3KOvsS3FL_FC'] = np.exp2(df.S3KOvsS3FL_Log2FC)

Now, we want to pull out the specific lysine residues acetylated on each individual peptide. We want these residue positions related back to the master protein (i.e., not the position within the peptide). This isn't easily achieved in how the PD data is currently exported. Some peptides could correspond to multiple protein accessions, each with different lysine positions. All possible accessions and positions are listed in the peptide output - however, the order is arbitrary. Therefor, we need to iterate though the accessions listed and match them back to the master protein accession. 

In [34]:
kac_sites = df['Modifications in Proteins'].str.split('; (?!K)', expand=True, )
kac_sites.head()

,0,1,2,3,4,5,6,7,8,9
225,O08756 1xAcetyl [K104(100)],A2AFQ2 1xAcetyl [K104(100)],Q99N15 1xAcetyl [K104(100)],None,None,None,None,None,None,None
146,Q8QZT1 1xAcetyl [K260(100)],None,None,None,None,None,None,None,None,None
20,Q8BMS1 1xAcetyl [K289(100)],None,None,None,None,None,None,None,None,None
7,Q8BMS1 2xAcetyl [K411(100); K414(100)],None,None,None,None,None,None,None,None,None
250,Q99JY0 1xAcetyl [K189(100)],None,None,None,None,None,None,None,None,None


In the above data, the peptide at index 225 is linked to the master protein accession Q99N15, located in column 3. In other rows with multiple accessions, the master protein accession can be found in any of the 10 columns listed. Therefore, we will iterate through the columns of these sites to match them back to the original accession. The matches will be combined into a list, and the list will be transformed into a Series

In [35]:
# list to hold the sites associated with the same master protein accession
sites = []
for i in np.arange(0, kac_sites.columns[-1]+1):
    bool_mask = df.Accession == kac_sites[i].str.split(' ', expand=True)[0]
    try:
        temp_sites = kac_sites[i].loc[bool_mask].str.split('(?<!;)\s', expand=True)[2]
        sites.append(temp_sites)
    except:
        continue

In [36]:
# convert the list into a pandas series     
sites = pd.concat(sites)

# remove the percent confidence and brackets 
sites = sites.str.replace('\(\d{1,6}\)', '').str.replace('\[(.+)\]', '\g<1>')

sites.rename('Acetylated Residues', inplace=True)

sites.head(10)

146          K260
20           K289
7      K411; K414
250          K189
293           K46
19           K406
166          K316
262           K70
292           K46
17           K350
Name: Acetylated Residues, dtype: object

Now, we add the specific sites back to the dataframe and prepare to export

In [37]:
# merge the residues back onto the dataframe
df = df.merge(sites.to_frame(), left_index=True, right_index=True)

In [38]:
export_cols = ['Accession', 'Name', 'Symbol', 'Gene Symbol', 'Entrez', 
               'Matrix', 'IMS', 'Acetylated Residues', 'DKOvsDFC_FC', 'S3KOvsS3FL_FC']


df[export_cols].head(25)

,Accession,Name,Symbol,Gene Symbol,Entrez,Matrix,IMS,Acetylated Residues,DKOvsDFC_FC,S3KOvsS3FL_FC
225,Q99N15,17beta-hydroxysteroid dehydrogenase type 10/sh...,Hsd17b10,Hsd17b10,15108,True,False,K104,608.826588,76.675257
146,Q8QZT1,"Acetyl-CoA acetyltransferase, mitochondrial",Acat1,Acat1,110446,True,False,K260,452.538272,117.104532
20,Q8BMS1,"Trifunctional enzyme subunit alpha, mitochondrial",Hadha,Hadha,97212,True,False,K289,323.774243,79.883697
7,Q8BMS1,"Trifunctional enzyme subunit alpha, mitochondrial",Hadha,Hadha,97212,True,False,K411; K414,240.042300,3.405044
250,Q99JY0,"Trifunctional enzyme subunit beta, mitochondrial",Hadhb,Hadhb,231086,False,False,K189,140.717881,31.524822
293,P03930,ATP synthase protein 8,Mtatp8,ATP8,17706,False,False,K46,117.415371,69.121741
19,Q8BMS1,"Trifunctional enzyme subunit alpha, mitochondrial",Hadha,Hadha,97212,True,False,K406,101.754690,103.153493
166,O35459,"Delta(3,5)-Delta(2,4)-dienoyl-CoA isomerase, m...",Ech1,Ech1,51798,True,False,K316,96.442740,9.748235
262,Q3ULD5,"Methylcrotonoyl-CoA carboxylase beta chain, mi...",Mccc2,Mccc2,78038,False,False,K70,81.816498,31.088696
292,P03930,ATP synthase protein 8,Mtatp8,ATP8,17706,False,False,K46,65.599156,77.843621


In [39]:
writer = pd.ExcelWriter('processed_files/Exp1_DKOvsSirt3_Top25.xlsx', options={'strings_to_numbers': True})

df[export_cols].sort_values('DKOvsDFC_FC', ascending=False).head(25).to_excel(writer, sheet_name='Exp1_Top25_Kac_DKOvsDFC', index=False)
df[export_cols].sort_values('S3KOvsS3FL_FC', ascending=False).head(25).to_excel(writer, sheet_name='Exp1_Top25_Kac_S3KOvsS3FL', index=False)

writer.save()


df[export_cols].sort_values('DKOvsDFC_FC', ascending=False).head(25).to_csv('processed_files/Exp1_Top25_Kac_DKOvsDFC.csv', index=False)
df[export_cols].sort_values('S3KOvsS3FL_FC', ascending=False).head(25).to_csv('processed_files/Exp1_Top25_Kac_S3KOvsS3FL.csv', index=False)
